In [33]:
!python -m pip install --quiet --upgrade pip
!pip install --quiet -U sagemaker
!pip install -U --quiet transformers
import torch
import transformers

In [34]:
!python -V
print(f"torch version: {torch.__version__}")
print(f"transformers version: {transformers.__version__}")

Python 3.6.13
torch version: 1.7.1
transformers version: 4.11.3


In [35]:
import os
import time
import sagemaker

In [36]:
# S3 directory where the model is stored
bucket = "ai-inference-env"
prefix = "sentence-similarity"
key = os.path.join(prefix, "model.tar.gz")
pretrained_model_data = "s3://{}/{}".format(bucket, key)
pretrained_model_data

's3://ai-inference-env/sentence-similarity/model.tar.gz'

In [37]:
!pygmentize code_sentence_similarity/inference.py

import re
import sys
import logging
import json
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
from test_cuda import test_cuda

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))


JSON_CONTENT_TYPE = "application/json"

# checking that cuda is available inside container
test_cuda()

MODEL_PATH = "/opt/ml/model/"

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)


def get_device():
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    return device


def text_cleaning(text_to_clean):
    text_to_clean = re.sub(r"[.?!]+", " ", text_to_clean)
    text_to_clean = " ".join(text_to_clean.split())
    return text_to_clean.lower().strip()


# Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    # extracting the last_hidden_state tensor
    token_embeddings = model_outpu

In [38]:
#instance_type = "local_gpu" # or "local"
instance_type = "ml.g4dn.xlarge"

In [39]:
role = sagemaker.get_execution_role()
sess = sagemaker.Session()

In [40]:
from sagemaker.pytorch.model import PyTorchModel

pytorch_model = PyTorchModel(
    model_data=pretrained_model_data,
    role=role,
    framework_version="1.8.1",
    source_dir="./code_sentence_similarity",
    py_version="py3",
    entry_point="inference.py",
)

In [41]:
predictor = pytorch_model.deploy(endpoint_name= "sentence-similarity", initial_instance_count=1, instance_type=instance_type)

-------------------!

In [42]:
predictor.serializer = sagemaker.serializers.JSONSerializer()
predictor.deserializer = sagemaker.deserializers.JSONDeserializer()

In [48]:
user_sentence = "I love going to the beach"
ground_truth = "the beach is beautiful"

In [50]:
result = predictor.predict({"user_input": user_sentence, "true_sentence": ground_truth})
print(result)
print(f"result: {result['similarity']}")

{'similarity': '0.6418977975845337'}
result: 0.6418977975845337


### Computes inference average time ###

In [45]:
inference_time = []
for _ in range(50):
    start = time.time()
    predictor.predict({"user_input": user_sentence, "true_sentence": ground_truth})
    inference_time.append(time.time()-start)
    
print(f"Average inference on GPU is: {sum(inference_time)/len(inference_time):.3} ms")

Average inference on GPU is: 0.0347 ms


### Clean Up ###

In [19]:
#predictor.delete_endpoint(predictor.endpoint)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
